# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.cpp_extension import load
import torch.nn.functional as F
from collections import OrderedDict
from time import sleep

In [2]:
import peephole_lstm_classes

c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:92: UserWarning: Error checking compiler version: Command '['c++']' returned non-zero exit status 1.
  warnings.warn('Error checking compiler version: {}'.format(error))
c:\users\0107w\anaconda3\envs\pytorch\lib\site-packages\torch\utils\cpp_extension.py:118: UserWarning: 

                               !! WARNING !!

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Your compiler (c++) may be ABI-incompatible with PyTorch!
Please use a compiler that is ABI-compatible with GCC 4.9 and above.
See https://gcc.gnu.org/onlinedocs/libstdc++/manual/abi.html.

See https://gist.github.com/goldsborough/d466f43e8ffc948ff92de7486c5216d6
for instructions on how to install GCC 4.9 or higher.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                              !! WARNING !!

  warnings.warn(ABI_INCOMPATIBILITY_WARNING.format(compiler))


In [15]:
dropout = 0.
class PeepholeNetworkCPP(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTMCell(129, 300)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTMCell(300, 300)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTMCell(300, 300)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        output = x.new_empty(x.size(0), x.size(1), 129)
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        for i, seq_batch in enumerate(x.transpose(0, 1)):
            hc1 = self.lstm0(seq_batch, hc1)
            #states = self.lstm0(seq_batch, states)
            hc2 = self.lstm1(F.dropout(hc1[0], p=0, training=self.training), hc2)
            hc3 = self.lstm2(F.dropout(hc2[0], p=0, training=self.training), hc3)
            output[:, i, :] = self.fc(F.dropout(hc3[0], p=0, training=self.training))
            #output[:, i, :] = self.fc(F.dropout(states[0], p=0, training=self.training))
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return output, new_states
        #return output, states
class PeepholeNetworkTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTMCellTorch(129, 300)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTMCellTorch(300, 300)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTMCellTorch(300, 300)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        output = x.new_empty(x.size(0), x.size(1), 129)
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        for i, seq_batch in enumerate(x.transpose(0, 1)):
            hc1 = self.lstm0(seq_batch, hc1)
            #states = self.lstm0(seq_batch, states)
            hc2 = self.lstm1(F.dropout(hc1[0], p=0, training=self.training), hc2)
            hc3 = self.lstm2(F.dropout(hc2[0], p=0, training=self.training), hc3)
            output[:, i, :] = self.fc(F.dropout(hc3[0], p=0, training=self.training))
            #output[:, i, :] = self.fc(F.dropout(states[0], p=0, training=self.training))
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return output, new_states
        #return output, states
class PeepholeLoopTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTMTorch(129, 300, 0)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTMTorch(300, 300, 0)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTMTorch(300, 300, 0)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states
class PeepholeLoop(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeLSTM(129, 300, 0)
        self.lstm1 = peephole_lstm_classes.PeepholeLSTM(300, 300, 0)
        self.lstm2 = peephole_lstm_classes.PeepholeLSTM(300, 300, 0)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        #x, states = self.lstm0(x, states)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states
        #return x, states
class PeepholeNormTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm0 = peephole_lstm_classes.PeepholeNormLSTMTorch(129, 300, 0)
        self.lstm1 = peephole_lstm_classes.PeepholeNormLSTMTorch(300, 300, 0)
        self.lstm2 = peephole_lstm_classes.PeepholeNormLSTMTorch(300, 300, 0)
        self.fc = nn.Linear(300, 129)
    def forward(self, x, states):
        hc1, hc2, hc3 = ((states[0][i], states[1][i]) for i in range(states[0].size(0)))
        x, hc1 = self.lstm0(x, hc1)
        x, hc2 = self.lstm1(x, hc2)
        x, hc3 = self.lstm2(x, hc3)
        x = self.fc(x)
        new_states = (torch.cat((hc1[0], hc2[0], hc3[0])), torch.cat((hc1[1], hc2[1], hc3[1])))
        return x, new_states


device = ['cpu', 'cuda:0'][1]

# model_cpp = peephole_lstm_cell_classes.PeepholeLSTMCell(129, 300)
# model_torch = peephole_lstm_cell_classes.PeepholeLSTMCellTorch(129, 300)
model_cpp = PeepholeNetworkCPP()
model_torch = PeepholeNetworkTorch()
model_torch_loop = PeepholeLoopTorch()
model_loop = PeepholeLoop()

model_norm_torch = PeepholeNormTorch()

for module_name in ('lstm0', 'lstm1', 'lstm2', 'fc'):
#for module_name in ('lstm0', 'fc'):
    copyfrom = getattr(model_cpp, module_name)
    if module_name != 'fc':
        mod = getattr(model_torch, module_name)
        mod.weight_ih.data = copyfrom.weight_ih.data
        mod.weight_hh.data = copyfrom.weight_hh.data
        mod.weight_ch.data = copyfrom.weight_ch.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_loop, module_name)
        mod.weight_ih.data = copyfrom.weight_ih.data
        mod.weight_hh.data = copyfrom.weight_hh.data
        mod.weight_ch.data = copyfrom.weight_ch.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_torch_loop, module_name)
        mod.weight_ih.data = copyfrom.weight_ih.data
        mod.weight_hh.data = copyfrom.weight_hh.data
        mod.weight_ch.data = copyfrom.weight_ch.data
        mod.bias.data = copyfrom.bias.data
    else:
        mod = getattr(model_torch, module_name)
        mod.weight.data = copyfrom.weight.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_loop, module_name)
        mod.weight.data = copyfrom.weight.data
        mod.bias.data = copyfrom.bias.data
        mod = getattr(model_torch_loop, module_name)
        mod.weight.data = copyfrom.weight.data
        mod.bias.data = copyfrom.bias.data

model_cpp.to(device)
model_torch.to(device)
model_torch_loop.to(device)
model_loop.to(device)
model_norm_torch.to(device)

batch_size = 32
seq_length = 5

play_input = torch.randn(batch_size, seq_length, 129).to(device)
play_states = (torch.zeros(3, batch_size, 300).to(device), torch.zeros(3, batch_size, 300).to(device))
#play_states = torch.zeros(4, 600).to(device).chunk(dim=1, chunks=2)

print(model_cpp)
print(model_torch)
print(model_torch_loop)
print(model_loop)
print(model_norm_torch)

PeepholeNetworkCPP(
  (lstm0): PeepholeLSTMCell(input_size=129, hidden_size=300)
  (lstm1): PeepholeLSTMCell(input_size=300, hidden_size=300)
  (lstm2): PeepholeLSTMCell(input_size=300, hidden_size=300)
  (fc): Linear(in_features=300, out_features=129, bias=True)
)
PeepholeNetworkTorch(
  (lstm0): PeepholeLSTMCellTorch(input_size=129, hidden_size=300)
  (lstm1): PeepholeLSTMCellTorch(input_size=300, hidden_size=300)
  (lstm2): PeepholeLSTMCellTorch(input_size=300, hidden_size=300)
  (fc): Linear(in_features=300, out_features=129, bias=True)
)
PeepholeLoopTorch(
  (lstm0): PeepholeLSTMTorch(input_size=129, hidden_size=300, dropout=0)
  (lstm1): PeepholeLSTMTorch(input_size=300, hidden_size=300, dropout=0)
  (lstm2): PeepholeLSTMTorch(input_size=300, hidden_size=300, dropout=0)
  (fc): Linear(in_features=300, out_features=129, bias=True)
)
PeepholeLoop(
  (lstm0): PeepholeLSTM(input_size=129, hidden_size=300, dropout=0.0)
  (lstm1): PeepholeLSTM(input_size=300, hidden_size=300, dropout=0

In [16]:
n0 = peephole_lstm_classes.PeepholeNormLSTMTorch(129, 300, dropout=0., momentum=0.1, eps=1e-8)
n1 = peephole_lstm_classes.NormalizedPeepholeLSTM(129, 300, dropout=0., momentum=0.1, eps=1e-8)

for params in zip(n0.parameters(), n1.parameters()):
    params[0].data = params[1].data

n0.cuda()
n1.cuda()

print(n0.running_var_tanh_cell)
print(n1.running_var_tanh_cell)
print("#####################################################")
print(n0(play_input, (play_states[0][0], play_states[1][0])))
print('-----------------------------------------------------')
print(n1(play_input, (play_states[0][0], play_states[1][0])))
print("#####################################################")
print(n0.running_var_tanh_cell)
print(n1.running_var_tanh_cell)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [17]:
n0.zero_grad()
n1.zero_grad()
p0, p1 = play_input.clone().requires_grad_(True), play_input.clone().requires_grad_(True)
noise = torch.randn(batch_size, 1, 300, device=device)

t = n0(p0, (play_states[0][0], play_states[1][0]))[0].mul(noise).sum()
print(t)
t.backward()
t = n1(p1, (play_states[0][0], play_states[1][0]))[0].mul(noise).sum()
print(t)
t.backward()
print()

print(n0.weight_ih.grad[::50])
print(n1.weight_ih.grad[::50])
print(p0.grad)
print(p1.grad)

tensor(92.4211, device='cuda:0', grad_fn=<SumBackward0>)
tensor(92.4211, device='cuda:0', grad_fn=<SumBackward0>)

tensor([[  0.0239,   0.0051,   0.0086,  ...,  -0.0319,  -0.0210,   0.0230],
        [ -0.7085,  -0.1210,  -0.7467,  ...,   0.1746,   0.2869,  -1.4869],
        [ -1.1944,  -0.9582,   0.5559,  ...,   1.1371,  -0.0159,  -0.2958],
        ...,
        [  2.8296,  -4.5818,  -4.4437,  ...,  -0.9316,  -0.8718,  -1.9201],
        [-35.2515,  39.3277, -41.2986,  ..., -46.2633,  33.2994,  43.5961],
        [-15.8175,  27.3627, -55.2537,  ..., -54.9299,  31.3079, -19.0987]],
       device='cuda:0')
tensor([[  0.0129,  -0.0129,   0.0072,  ...,  -0.0177,  -0.0159,   0.0125],
        [ -0.1319,  -0.2779,  -0.0901,  ...,   0.1413,   0.5245,  -0.1221],
        [ -0.4763,  -0.9574,  -0.1017,  ...,   0.7848,   0.1059,   0.5132],
        ...,
        [  0.3380,  -0.0357,  -1.9727,  ...,  -0.7785,   0.3042,   0.6908],
        [  2.8996,   6.2643,  -9.8068,  ..., -11.4440,  11.8460,  34.7389]

In [16]:
%%timeit -n 1 -r 50
n0(play_input, (play_states[0][0], play_states[1][0]))

176 ms ± 42.8 ms per loop (mean ± std. dev. of 50 runs, 1 loop each)


In [17]:
%%timeit -n 1 -r 50
n1(play_input, (play_states[0][0], play_states[1][0]))

386 ms ± 30.2 ms per loop (mean ± std. dev. of 50 runs, 1 loop each)


In [18]:
print(list(name for name, _ in n0.named_parameters()))
print(list(name for name, _ in n1.named_parameters()))

['weight_ih', 'weight_hh', 'weight_ch', 'bias', 'gamma_ih', 'gamma_hh', 'gamma_ch', 'gamma_tanh_cell', 'beta_tanh_cell']
['weight_ih', 'weight_hh', 'weight_ch', 'bias', 'gamma_ih', 'gamma_hh', 'gamma_ch', 'gamma_tanh_cell', 'beta_tanh_cell']


Vanila PyTorch implementation

In [4]:
%%timeit -n 1 -r 30
out = model_torch(play_input, play_states)[0]

493 ms ± 50.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [5]:
%%timeit -n 1 -r 30 out = model_torch(play_input, play_states)[0]
model_torch.zero_grad()
out.pow(2).sum().backward()

896 ms ± 24.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
model_torch.eval()
model_torch.zero_grad()
out = model_torch(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_torch.lstm0.weight_ih.grad.data.sum().add(model_torch.lstm0.weight_hh.grad.data.sum()).add(model_torch.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [7]:
sleep(3)
torch.cuda.empty_cache()

Loop within each module

In [8]:
%%timeit -n 1 -r 30
out = model_torch_loop(play_input, play_states)[0]

404 ms ± 19.2 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [9]:
%%timeit -n 1 -r 30 out = model_torch_loop(play_input, play_states)[0]
model_torch_loop.zero_grad()
out.pow(2).sum().backward()

896 ms ± 34.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [10]:
model_torch_loop.eval()
model_torch_loop.zero_grad()
out = model_torch_loop(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_torch_loop.lstm0.weight_ih.grad.data.sum().add(model_torch_loop.lstm0.weight_hh.grad.data.sum()).add(model_torch_loop.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [11]:
sleep(3)
torch.cuda.empty_cache()

Single Peephole LSTM Cell C++ (Loop implemented with Python)

In [12]:
%%timeit -n 1 -r 30
out = model_cpp(play_input, play_states)[0]

418 ms ± 29.8 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [13]:
%%timeit -n 1 -r 30 out = model_cpp(play_input, play_states)[0]
model_cpp.zero_grad()
out.pow(2).sum().backward()

805 ms ± 29.9 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [14]:
model_cpp.eval()
model_cpp.zero_grad()
out = model_cpp(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_cpp.lstm0.weight_ih.grad.data.sum().add(model_cpp.lstm0.weight_hh.grad.data.sum()).add(model_cpp.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [15]:
sleep(3)
torch.cuda.empty_cache()

For-Loop implemented inside C++

Custom Kernal

In [16]:
%%timeit -n 1 -r 30
out = model_loop(play_input, play_states)[0]

65.7 ms ± 7.98 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [17]:
%%timeit -n 1 -r 30 out = model_loop(play_input, play_states)[0]
model_loop.zero_grad()
out.pow(2).sum().backward()

51.3 ms ± 1.73 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [18]:
model_loop.eval()
model_loop.zero_grad()
out = model_loop(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_loop.lstm0.weight_ih.grad.data.sum().add(model_loop.lstm0.weight_hh.grad.data.sum()).add(model_loop.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[ 0.0562,  0.0598,  0.0121,  ..., -0.0333, -0.0352, -0.0040],
         [ 0.0576,  0.0620,  0.0103,  ..., -0.0361, -0.0360, -0.0049],
         [ 0.0576,  0.0631,  0.0094,  ..., -0.0379, -0.0359, -0.0053],
         ...,
         [ 0.0582,  0.0649,  0.0088,  ..., -0.0394, -0.0347, -0.0060],
         [ 0.0579,  0.0650,  0.0090,  ..., -0.0392, -0.0347, -0.0060],
         [ 0.0577,  0.0653,  0.0091,  ..., -0.0389, -0.0349, -0.0060]],

        [[ 0.0558,  0.0600,  0.0130,  ..., -0.0329, -0.0354, -0.0040],
         [ 0.0575,  0.0622,  0.0111,  ..., -0.0361, -0.0365, -0.0049],
         [ 0.0582,  0.0637,  0.0100,  ..., -0.0380, -0.0373, -0.0052],
         ...,
         [ 0.0575,  0.0636,  0.0074,  ..., -0.0397, -0.0349, -0.0064],
         [ 0.0578,  0.0634,  0.0074,  ..., -0.0398, -0.0345, -0.0064],
         [ 0.0578,  0.0635,  0.0078,  ..., -0.0394, -0.0345, -0.0066]],

        [[ 0.0560,  0.0601,  0.0124,  ..., -0.0335, -0.0353, -0.0036],
         [ 0.0578,  0.0626,  0.0106,  ..., -

In [19]:
sleep(3)
torch.cuda.empty_cache()

Normalized - Torch

In [4]:
%%timeit -n 1 -r 30
out = model_norm_torch(play_input, play_states)[0]

487 ms ± 60.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [5]:
%%timeit -n 1 -r 30 out = model_norm_torch(play_input, play_states)[0]
model_norm_torch.zero_grad()
out.pow(2).sum().backward()

918 ms ± 44.6 ms per loop (mean ± std. dev. of 30 runs, 1 loop each)


In [6]:
model_norm_torch.eval()
model_norm_torch.zero_grad()
out = model_norm_torch(play_input, play_states)
print(out)
out[0].pow(2).mean().backward()
print(model_norm_torch.lstm0.weight_ih.grad.data.sum().add(model_norm_torch.lstm0.weight_hh.grad.data.sum()).add(model_norm_torch.lstm0.weight_ch.grad.data.sum()).mul(10000))

(tensor([[[-0.0704, -0.0244,  0.0305,  ..., -0.0407,  0.0059, -0.0575],
         [-0.0381,  0.0100,  0.0460,  ..., -0.0236,  0.0547, -0.0603],
         [-0.0108,  0.0340,  0.0947,  ...,  0.0006,  0.0875, -0.0402],
         ...,
         [-0.2807, -0.1009,  0.0343,  ...,  0.0449, -0.1193, -0.0647],
         [-0.2771, -0.0293, -0.0023,  ...,  0.0099, -0.1284, -0.1171],
         [-0.3189, -0.0023, -0.0334,  ..., -0.0255, -0.0703, -0.1567]],

        [[-0.0681, -0.0217,  0.0038,  ..., -0.0490,  0.0505, -0.0682],
         [-0.1424, -0.0186, -0.0091,  ..., -0.0620,  0.0914, -0.0968],
         [-0.1891,  0.0523, -0.0042,  ..., -0.0220,  0.0909, -0.0717],
         ...,
         [ 0.2524,  0.0960,  0.3093,  ...,  0.0566,  0.0381, -0.1388],
         [ 0.2102,  0.0286,  0.2966,  ...,  0.0631,  0.0219, -0.1538],
         [ 0.1145, -0.0168,  0.2166,  ...,  0.0050, -0.0295, -0.1838]],

        [[-0.0538, -0.0228,  0.0727,  ..., -0.0062,  0.0874, -0.0723],
         [-0.0777, -0.0269,  0.0551,  ...,  

In [7]:
sleep(3)
torch.cuda.empty_cache()

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

In [3]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

And we can see those same characters from above, encoded as integers.

In [6]:
encoded[:100]

array([55, 23, 78, 82, 41, 13, 65, 58, 80, 34, 34, 34, 25, 78, 82, 82, 72,
       58, 75, 78, 50, 14,  8, 14, 13,  3, 58, 78, 65, 13, 58, 78,  8,  8,
       58, 78,  8, 14, 42, 13, 59, 58, 13, 15, 13, 65, 72, 58, 11, 68, 23,
       78, 82, 82, 72, 58, 75, 78, 50, 14,  8, 72, 58, 14,  3, 58, 11, 68,
       23, 78, 82, 82, 72, 58, 14, 68, 58, 14, 41,  3, 58, 67,  6, 68, 34,
        6, 78, 72, 73, 34, 34,  7, 15, 13, 65, 72, 41, 23, 14, 68])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, that we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ batches. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences in a batch, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. We also want to create both the input and target arrays. Remember that the targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
#     n_batches = ((len(arr) - batch_size) // seq_length) // batch_size
    n_batches = ((len(arr) - 1) // seq_length) // batch_size
    
    ## TODO: Keep only enough characters to make full batches
#     arr = arr[:batch_size * (seq_length * n_batches + 1)]
    arr = np.concatenate([arr[seq * seq_length * n_batches:(seq + 1) * seq_length * n_batches + 1]
                          for seq in range(batch_size)])
    
    ## TODO: Reshape into batch_size rows
    arr = arr.reshape((batch_size, seq_length * n_batches + 1))
    
    ## TODO: Iterate over the batches using a window of size seq_length
#     for n in range(0, arr.shape[1], seq_length):
    for n in range(0, arr.shape[1] - 1, seq_length):
        # The features
        x = arr[:, n:n +  seq_length]
        # The targets, shifted by one
        y = arr[:, n + 1:n + seq_length + 1]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 8 and 50 sequence steps.

In [10]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[55 23 78 82 41 13 65 58 80 34]
 [ 3 67 68 58 41 23 78 41 58 78]
 [13 68 79 58 67 65 58 78 58 75]
 [ 3 58 41 23 13 58 49 23 14 13]
 [58  3 78  6 58 23 13 65 58 41]
 [49 11  3  3 14 67 68 58 78 68]
 [58 48 68 68 78 58 23 78 79 58]
 [29 53  8 67 68  3 42 72 73 58]]

y
 [[23 78 82 41 13 65 58 80 34 34]
 [67 68 58 41 23 78 41 58 78 41]
 [68 79 58 67 65 58 78 58 75 67]
 [58 41 23 13 58 49 23 14 13 75]
 [ 3 78  6 58 23 13 65 58 41 13]
 [11  3  3 14 67 68 58 78 68 79]
 [48 68 68 78 58 23 78 79 58  3]
 [53  8 67 68  3 42 72 73 58 17]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 although the exact numbers may be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial hidden state of all zeros. This is done like so

```python
self.init_hidden()
```

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')
# train_on_gpu = False

Training on GPU!


In [13]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        self.lstm = nn.LSTM(input_size=len(tokens), hidden_size=n_hidden, num_layers=n_layers,
                            batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(in_features=n_hidden, out_features=len(tokens))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0., lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the layers of the model
        in_size = len(tokens)
        for i in range(n_layers):
            self.add_module(f'lstm{i}', peephole_lstm_classes.PeepholeLSTM(in_size, n_hidden, drop_prob))
            in_size = n_hidden
        #self.lstm = peephole_lstm_classes.PeepholeLSTM(len(tokens), n_hidden, drop_prob)
        self.fc = nn.Linear(in_features=in_size, out_features=len(tokens))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        new_hidden = (torch.zeros_like(hidden[0]), torch.zeros_like(hidden[1]))
        for i in range(self.n_layers):
            x, hc = self.__getattr__(f'lstm{i}')(x, (hidden[0][i], hidden[1][i]))
            new_hidden[0][i], new_hidden[1][i] = hc[0], hc[1]
        x = self.fc(x)
        
        # return the final output and the hidden state
        return x, new_hidden
    
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new_zeros(self.n_layers, batch_size, self.n_hidden).cuda(),
                      weight.new_zeros(self.n_layers, batch_size, self.n_hidden).cuda())
        else:
            hidden = (weight.new_zeros(self.n_layers, batch_size, self.n_hidden),
                      weight.new_zeros(self.n_layers, batch_size, self.n_hidden))
        
        return hidden
        

In [15]:
class MyCharRNN(nn.Module):
    
    class PeepholeLSTMCell(nn.Module):
        
        def __init__(self, input_size, hidden_size, dropout=0):
            assert isinstance(input_size, int) and input_size >= 1
            assert isinstance(hidden_size, int) and hidden_size >= 1
            assert isinstance(dropout, (int, float)) and 0 <= dropout <= 1

            super().__init__()

            self.input_size = input_size
            self.hidden_size = hidden_size
            self.dropout = dropout
            
            self.weight_ih = nn.Parameter(torch.empty(4*hidden_size, input_size))
            self.weight_hh = nn.Parameter(torch.empty(4*hidden_size, hidden_size))
            self.weight_ch = nn.Parameter(torch.empty(3*hidden_size, hidden_size))
            self.bias = nn.Parameter(torch.empty(4*hidden_size))
            
            self.reset_parameters()
            
        def reset_parameters(self):
            stdv = 1. / np.sqrt(self.input_size + 2 * self.hidden_size)
            self.weight_ih.data.uniform_(-stdv, stdv)
            self.weight_hh.data.uniform_(-stdv, stdv)
            self.weight_ch.data.uniform_(-stdv, stdv)
            self.bias.data.uniform_(-stdv, stdv)
            
        def forward(self, input, hc=None):
            if hc is None:
                hc = input.new_zeros(input.size(1), 2 * self.hidden_size, requires_grad=False).chunk(chunks=2, dim=1)

            for i, h in enumerate((hc[0], hc[1])):
                if input.size(0) != h.size(0):
                    raise RuntimeError(f"Input batch size {input.size(0)} doesn't match hidden{i} batch size {h.size(0)}")
                if h.size(1) != self.hidden_size:
                    raise RuntimeError(f"hidden{i} has inconsistent hidden_size: got {h.size(1)}, expected {self.hidden_size}")
            
            gates = torch.addmm(self.bias, input, self.weight_ih.t())
            gates += torch.mm(hc[0], self.weight_hc.t())
            gates[:, :3*self.hidden_size] += torch.mm(hc[1], self.weight_ch.t())
            
            sig_gates = gates[:, :3 * self.hidden_size].sigmoid().chunk(chunks=3, dim=1)
            tanh_gate = gates[:, 3 * self.hidden_size:].tanh()
            
            c = sig_gates[0] * hc[1] + sig_gates[1] * tanh_gate
            h = sig_gates[2] * torch.tanh(c)
            output = F.dropout(h, p=self.dropout, training=self.training, inplace=False)
            
            return output, (h, c)
            
        def __repr__(self):
            return f"PeepholeLSTMCell(input_size={self.input_size}, hidden_size={self.hidden_size}, dropout={self.dropout})"
        
        def extra_repr(self):
            return ''

    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the layers of the model
        self.peephole_lstm0 = self.PeepholeLSTMCell(input_size=len(tokens), hidden_size=n_hidden, dropout=drop_prob)
        for n in range(1, n_layers):
            self.add_module(f"peephole_lstm{n}", self.PeepholeLSTMCell(input_size=n_hidden, hidden_size=n_hidden, dropout=drop_prob))
        self.fc = nn.Linear(in_features=n_hidden, out_features=len(tokens))
    
    def forward(self, x, hidden=None):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        if len(x.size()) != 3:
            raise RuntimeError(f"Invalid dimensions {x.size()} for the input tensor. "
                               "A 3D tensor with dimensions [batch_size, sequence_length, n_features] is expected.")
        if hidden is None:
            hidden = x.new_zeros(self.n_layers, x.size(0), 2 * self.hidden_size, requires_grad=False).chunk(chunks=2, dim=2)
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        out = x.transpose(0, 1)
        new_hidden = torch.empty_like(hidden[0])
        new_cell = torch.empty_like(hidden[1])
        for l in range(self.n_layers):
            outs = []
            h = hidden[0][l]
            c = hidden[1][l]
            for step_batch in out:
                out, (h, c) = self.__getattr__(f"peephole_lstm{l}")(step_batch, (h, c))
                outs.append(out)
            out = torch.stack(outs)
            new_hidden[l] = h
            new_cell[l] = c

        out = out.transpose(0, 1).contiguous()

        out = self.fc(out)
        hidden = (new_hidden, new_cell)
        
        # return the final output and the hidden state
        return out, hidden
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden


In [13]:
module = peephole_lstm_classes.PeepholeLayerNormLSTMTorch
# module = peephole_lstm_classes.PeepholeNormLSTMTorch
class MyCharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the layers of the model
        self.peephole_lstm0 = module(input_size=len(tokens), hidden_size=n_hidden, dropout=drop_prob)
        for n in range(1, n_layers):
            self.add_module(f"peephole_lstm{n}", module(input_size=n_hidden, hidden_size=n_hidden, dropout=drop_prob))
        self.fc = nn.Linear(in_features=n_hidden, out_features=len(tokens))
    
    def forward(self, x, hidden=None):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        if len(x.size()) != 3:
            raise RuntimeError(f"Invalid dimensions {x.size()} for the input tensor. "
                               "A 3D tensor with dimensions [batch_size, sequence_length, n_features] is expected.")
        if hidden is None:
            hidden = x.new_zeros(self.n_layers, x.size(0), 2 * self.hidden_size, requires_grad=False).chunk(chunks=2, dim=2)
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        new_hidden = torch.empty_like(hidden[0])
        new_cell = torch.empty_like(hidden[1])
        for l in range(self.n_layers):
            x, (new_hidden[l], new_cell[l]) = self.__getattr__(f"peephole_lstm{l}")(x, (hidden[0][l], hidden[1][l]))

        x = self.fc(x)

        # return the final output and the hidden state
        return x, (new_hidden, new_cell)
    
    def init_hidden(self, batch_size, cuda=True):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if cuda:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden


## Time to train

The train function gives us the ability to set the number of epochs, the learning rate, and other parameters.

Below we're using an Adam optimizer and cross entropy loss since we are looking at character class scores as output. We calculate the loss and perform backpropagation, as usual!

A couple of details about training: 
>* Within the batch loop, we detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [14]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if (train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size, train_on_gpu)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output.view(batch_size*seq_length, output.shape[-1]), targets.contiguous().view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
#                 # Get validation loss
#                 val_h = net.init_hidden(batch_size, train_on_gpu)
#                 val_losses = []
#                 net.eval()
#                 for x, y in get_batches(val_data, batch_size, seq_length):
#                     # One-hot encode our data and make them Torch tensors
#                     x = one_hot_encode(x, n_chars)
#                     x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
#                     # Creating new variables for the hidden state, otherwise
#                     # we'd backprop through the entire training history
#                     val_h = tuple([each.data for each in val_h])
                    
#                     inputs, targets = x, y
#                     if(train_on_gpu):
#                         inputs, targets = inputs.cuda(), targets.cuda()

#                     output, val_h = net(inputs, val_h)
#                     val_loss = criterion(output.view(batch_size*seq_length, output.shape[-1]), targets.contiguous().view(batch_size*seq_length).long())
                
#                     val_losses.append(val_loss.item())
                
#                 net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()))
#                       "Val Loss: {:.4f}".format(np.mean(val_losses)))
    torch.cuda.empty_cache()

## Instantiating the model

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [15]:
## TODO: set you model hyperparameters
# define and print the net
n_hidden=300
n_layers=3

# net = CharRNN(chars, n_hidden, n_layers, drop_prob=0.5, lr=0.0001)
net = MyCharRNN(chars, n_hidden, n_layers, drop_prob=0.5, lr=0.001)
print(net)

MyCharRNN(
  (peephole_lstm0): PeepholeLayerNormLSTMTorch(input_size=83, hidden_size=300, dropout=0.5, eps=1e-08)
  (peephole_lstm1): PeepholeLayerNormLSTMTorch(input_size=300, hidden_size=300, dropout=0.5, eps=1e-08)
  (peephole_lstm2): PeepholeLayerNormLSTMTorch(input_size=300, hidden_size=300, dropout=0.5, eps=1e-08)
  (fc): Linear(in_features=300, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [16]:
batch_size = 32
seq_length = 500
n_epochs = 1 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, clip=1, print_every=1)

Epoch: 1/1... Step: 1... Loss: 4.4687...
Epoch: 1/1... Step: 2... Loss: 4.3059...
Epoch: 1/1... Step: 3... Loss: 4.0469...
Epoch: 1/1... Step: 4... Loss: 3.8737...
Epoch: 1/1... Step: 5... Loss: 3.7228...
Epoch: 1/1... Step: 6... Loss: 3.6079...
Epoch: 1/1... Step: 7... Loss: 3.5097...
Epoch: 1/1... Step: 8... Loss: 3.4352...
Epoch: 1/1... Step: 9... Loss: 3.3555...
Epoch: 1/1... Step: 10... Loss: 3.3152...
Epoch: 1/1... Step: 11... Loss: 3.2409...
Epoch: 1/1... Step: 12... Loss: 3.2436...
Epoch: 1/1... Step: 13... Loss: 3.2251...
Epoch: 1/1... Step: 14... Loss: 3.2227...
Epoch: 1/1... Step: 15... Loss: 3.1817...
Epoch: 1/1... Step: 16... Loss: 3.2073...
Epoch: 1/1... Step: 17... Loss: 3.1983...
Epoch: 1/1... Step: 18... Loss: 3.1946...
Epoch: 1/1... Step: 19... Loss: 3.1660...
Epoch: 1/1... Step: 20... Loss: 3.1667...
Epoch: 1/1... Step: 21... Loss: 3.1682...
Epoch: 1/1... Step: 22... Loss: 3.1755...
Epoch: 1/1... Step: 23... Loss: 3.2026...
Epoch: 1/1... Step: 24... Loss: 3.1603...
E

KeyboardInterrupt: 

In [26]:
sleep(3)
torch.cuda.empty_cache()

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [17]:
# change the name, for saving multiple files
model_name = 'peeprnn.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out.view(-1, out.shape[-1]), dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='Anna ', top_k=5))

Anna eeo a eot tto oe  e eet teooe ee o ote  tee a et at  oee aete eaot  tt  et eaetoo ea aeo t   oet     eaeae  t ettoa  e  aaa att  eo ooo oett eet to ttee o e aa  a aeooetaoata  e aatea teo  a o  a ttteto ee ttoeaeet t aoeae aetta  ooo a e oa a a aotta   to ea  toae aoot ao  a o  a eete te to ae oa oott a tt  eaet   e aa teete aa  aot  oo  ae ea  ee ot o ta ea    ea    t  ae eaeo t oa  t eo eea e ae eaoaoeetet   ae ea aot at a tee oe oeo e ea aaet  et   o  oe oeee tt  eeatoataoe ae   o   t ooeeet  o oo  otoetaaato aa e tte  att tt  o  aa eoo  eoee t ee t et e teto oaeoo    t    aet to  at o tt et eaet   o  o e ea oa a  e t aa  oa tae   e a aet t   o et o o a eaaa tat ea a a    t  ao   t et     t toea ae e  e att aeae atooee a aetteeee   oe ot o t oet  oao a oeet o   oee   ot  t  ao ae ee ote aoe ot aootat o o oe ata a a totoee  etoe et ote oae aot a  te a te teaae oa aeaa t    oa a etteaet  eee  o toaa ee ot ee eea eo oo   eeto a aae  t taae  oeoe   eoa a otttt  oatt etee  oe e  t  

## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))